# Obtaining data for MA-DPG evaluation form example 02b


In [169]:
# Module imports
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine

# assume module imports
import examples.examples as examples
from assume import World
from assume.scenario.loader_csv import load_file, load_scenario_folder, run_learning

## 1. Running example 02b

In [150]:
example = "small_learning_2"
db_uri = "sqlite:///../local_db/assume_db.db"
inputs_dir = "../inputs"
scenario = examples.available_examples[example]["scenario"]
study_case = examples.available_examples[example]["study_case"]

In [ ]:
# Run example 02b
world = World(database_uri=db_uri, export_csv_path="../" + examples.csv_path)
load_scenario_folder(world, inputs_dir, scenario, study_case)
run_learning(
    world,
    inputs_dir,
    scenario,
    study_case,
)
world.run()

## 2. Retrieving the actions of the actors from the best run

In [ ]:
# Best actors directory
best_actors_dir = os.path.join(
    inputs_dir,
    scenario,
    "learned_strategies",
    study_case,
    "avg_reward_eval_policies/actors/",
)
actors = os.listdir(best_actors_dir)
actors

### 2.1 Option 1: Retrieving the actions from the database:

In [153]:
# Get the config file
config_path = os.path.join(inputs_dir, scenario, "config.yaml")

# Read the number of validation episodes from the config file
with open(config_path) as file:
    config = yaml.safe_load(file)[study_case]
learning_config = config["learning_config"]
no_of_val_episodes = (
    learning_config["training_episodes"]
    - learning_config["episodes_collecting_initial_experience"]
) // learning_config.get("validation_episodes_interval", 5)

In [ ]:
# Set up the database connection
db = create_engine(db_uri)
simulation = f"{scenario}_{study_case}_eval"

# Get the average reward for each episode in order to determine the best episode.
reward_df = pd.DataFrame(columns=["avg_reward"], index=range(1, no_of_val_episodes + 1))
for episode in range(1, no_of_val_episodes + 1):
    query = f"SELECT AVG(reward) as avg_reward FROM rl_params where simulation = '{simulation}_{episode}'"
    reward_df.at[episode, "avg_reward"] = pd.read_sql(query, db).values[0][0]
reward_df.head()

In [ ]:
episode = reward_df["avg_reward"].idxmax()
query = f"SELECT datetime as dt, unit, actions_0, actions_1 FROM rl_params where simulation = '{simulation}_{episode}'"
actions_df = pd.read_sql(query, db)
actions_df.index = pd.to_datetime(actions_df["dt"])
actions_df.drop(columns=["dt"], inplace=True)
actions_df

## 3. Getting the demand dataframe

In [ ]:
start = pd.Timestamp(config["start_date"])
end = pd.Timestamp(config["end_date"])

index = pd.date_range(
    start=start,
    end=end,
    freq=config["time_step"],
)

demand_df = load_file(
    os.path.join(inputs_dir, scenario), config, file_name="demand_df", index=index
)
demand_df

In [ ]:
pp_units = pd.read_csv(
    os.path.join(inputs_dir, scenario, "powerplant_units.csv"), index_col=0
)
pp_units

In [ ]:
df = pd.merge(demand_df, actions_df, left_index=True, right_on="dt")
df = df.set_index("unit", append=True)
df

In [ ]:
# sample 10% of the datetime indices
dates = df.index.get_level_values("dt").unique()
sampled_dates = pd.Index(dates).to_series().sample(frac=0.1, random_state=42)
sample_df = df.loc[df.index.get_level_values("dt").isin(sampled_dates)]
rest_df = df.loc[~df.index.get_level_values("dt").isin(sampled_dates)]
sample_df

In [ ]:
colors = list(["green"] * len(rest_df)) + list(["blue"] * len(sample_df))

# Scatter matrix
fig = pd.plotting.scatter_matrix(
    pd.concat([rest_df, sample_df], sort=False),
    c=colors,
    figsize=(7, 7),
    range_padding=0.2,
    hist_kwds={"bins": 20},  # Generic histogram configuration
    s=30,
    alpha=0.5,
)

# Customize histogram colors for each diagonal
hist_colors = ["green", "blue"]
for i, ax in enumerate(fig.diagonal()):
    data_combined = pd.concat([rest_df.iloc[:, i], sample_df.iloc[:, i]])
    ax.hist(
        [rest_df.iloc[:, i], sample_df.iloc[:, i]],
        bins=20,
        color=hist_colors,
        stacked=True,
        alpha=0.7,
    )

# Show plot
plt.show()

In [161]:
demand_df_all_year = pd.read_csv(
    os.path.join(inputs_dir, scenario, "demand_df.csv"), index_col=0
)
demand_df_all_year.index = pd.to_datetime(demand_df_all_year.index)

In [ ]:
# Add a 'season' column
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"


demand_df_all_year["season"] = demand_df_all_year.index.get_level_values(
    "datetime"
).month.map(get_season)
demand_df = demand_df_all_year
# demand_df['season'] = demand_df.index.get_level_values('dt').month.map(get_season)

# Group data by season
seasons = ["Winter", "Spring", "Summer", "Fall"]
season_colors = {"Winter": "blue", "Spring": "green", "Summer": "orange", "Fall": "red"}
season_data = {
    season: demand_df[demand_df["season"] == season]["demand_EOM"] for season in seasons
}

# Plot histogram with different colors for each season
plt.figure(figsize=(10, 6))
plt.hist(
    [season_data[season] for season in seasons],
    bins=20,
    stacked=True,
    color=[season_colors[season] for season in seasons],
    label=seasons,
    alpha=0.7,
)
plt.xlabel("Demand (EOM)")
plt.ylabel("Frequency")
plt.title("Histogram of Demand EOM by Season")
plt.legend(title="Season")
plt.show()

Leichter demand shift zum Winter hin, zu erklären durch Heizwärmebedarf

In [ ]:
scaler = StandardScaler()
scaled_rest = scaler.fit_transform(rest_df)
scaled_sample = scaler.transform(sample_df)

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
tsne_results = tsne.fit_transform(np.append(scaled_rest, scaled_sample, axis=0))

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=colors, alpha=0.7)
plt.title("t-SNE Dimensionality Reduction")
plt.xlabel("tSNE-1")
plt.ylabel("tSNE-2")
plt.show()